Lấy SummonerID

In [5]:
import time
import requests
api_key = 'RGAPI-415a160c-532f-41e2-9a53-78299b01cb10'


In [ ]:

rank = 'CHALLENGER'
url = f"https://vn2.api.riotgames.com/lol/league-exp/v4/entries/RANKED_SOLO_5x5/{rank}/I?page=1&api_key={api_key}"
response = requests.get(url)
if response.status_code == 200:
    data = response.json()
    if data:
        for summoner in data:
            summonerID = summoner.get("summonerId")
            with open ("ChallengerSummonerID.txt", "a+") as file:
                file.write(summonerID + "\n")
else:
    print("Failed to retrieve. Status Code:", response.status_code)

Lấy PUUID

In [ ]:
file_path = 'ChallengerSummonerID.txt'
with open(file_path, 'r') as file:
    for line in file:
        summoner_ID = line.strip()
        url1 = f"https://vn2.api.riotgames.com/lol/summoner/v4/summoners/{summoner_ID}?api_key={api_key}"
        response = requests.get(url1)
        if response.status_code == 200:
            data = response.json()
            player_puuid = data.get('puuid')
            with open('ChallengerPlayerPUUID.txt', 'a+') as file:
                file.write(player_puuid + "\n")
        else:
            print("Failed to retrieve. Status Code:", response.status_code)


Lấy MatchID

In [ ]:
file_path1 = 'ChallengerPlayerPUUID.txt'
with open(file_path1, 'r') as file:
    for line in file:
        PUUID = line.strip()
        url = f"https://sea.api.riotgames.com/lol/match/v5/matches/by-puuid/{PUUID}/ids?queue=420&type=ranked&start=0&count=20&api_key={api_key}"
        success = False
        while not success:
            response = requests.get(url)
            if response.status_code == 200:
                matchID = response.json()
                with open('ChallengerPlayerMatchID.txt', 'a+') as output_file:
                    output_file.write(str(matchID) + "\n")
                
                success = True
                time.sleep(6)
            else:
                print(f"Failed to retrieve data for PUUID. Status Code: {response.status_code}.")
                time.sleep(5)
            

Lấy các thông tin từ 1 Match: kills, deaths, assists, kda, total golds, total minions

In [95]:
match_id = "VN2_629235327"
url3 = f'https://sea.api.riotgames.com/lol/match/v5/matches/{match_id}?api_key={api_key}'
response3 = requests.get(url3)
if response3.status_code == 200:
    match_info = response3.json()
    participants = match_info['metadata']['participants']
    target_string = "40EdfkFWiwvYDq3hQ-VH2MjJIhdmEYWfuhfm_7MnG6WGxYxagQMGDtNmbO5f7YsLCyKtbaYAdPHfsQ"
    if target_string in participants:
        index_of_target = participants.index(target_string)
    else:
        print("Not found")
    participant_info = match_info['info']['participants'][index_of_target]
    kills = participant_info['kills']
    deaths = participant_info['deaths']
    assists = participant_info['assists']
    if deaths == 0:
        kda = kills + assists 
    else:
        kda = (kills + assists) / deaths
    total_gold = participant_info['goldEarned']
    total_minion = participant_info['totalMinionsKilled']
    with open('PlayerStats.txt', 'a+') as output_file:
            output_file.write(f"Kills: {kills}, Deaths: {deaths}, Assists: {assists}, KDA: {kda}, Gold: {total_gold}, Minions: {total_minion}\n")
else:
    print("Failed to retrieve. Status Code:", response3.status_code)